<div
  style="
    display: flex;
    justify-content: center;
    flex-direction: column;
    align-items: center;
  "
>
<div style="width: 400px; padding: 20px; background: #111; border: 1px solid #333; color: #eee">
  <p style="opacity: 0.5; margin-bottom: -10px"><i>TDT4173 Machine Learning 2023</i></p>
  <h1 style="margin-bottom: -5px"><b>Short Notebook 2</b></h1>
  <h3>Kaggle Name: NeuralNet Ninjas</h3>
  <br />
  <h3>Team Members:</h3>
  <table style="margin: 0 auto; width: 100%; text-align: left">
    <tr style="background: #222">
      <th style="border-width: 0.5px; border-color: #555">Name</th>
      <th style="border-width: 0.5px; border-color: #555">Student ID</th>
    </tr>
    <tr>
      <td style="border-width: 0.5px; border-color: #555">Antonsen Eggen, Sivert</td>
      <td style="border-width: 0.5px; border-color: #555">123 45 678</td>
    </tr>
    <tr>
      <td style="border-width: 0.5px; border-color: #555">Broch Grude, Kristoffer VI Nicolay</td>
      <td style="border-width: 0.5px; border-color: #555">123 45 678</td>
    </tr>
    <tr>
      <td style="border-width: 0.5px; border-color: #555">Raa, Mathias</td>
      <td style="border-width: 0.5px; border-color: #555">123 45 678</td>
    </tr>
  </table>
  <br />
  <h3 style="margin-bottom: -5px">Runtime: x hours</h3>
  <p style="opacity: 0.5;"><i>Specs: MacBook Pro 2021, Apple M1 Pro Chip, 16 GB RAM</i></p>
  </div>
</div>

---

In [ ]:
# Dictionary of all model predictions for final ensambling
test_result = {}

# 1 – Preprocessing

# 2 – Modeling

**Model Stack**
1. Target Encoded Catboost
2. Simple LightGBM
3. Target Log Transform Catboost
4. Location Specific Catboost
5. Location Specific AutoGluon
6. Location Specific FastAI Neural Network

## Model 1 – Target Encoded Catboost

In [ ]:
data_general_catboost_transform = data_catboost.copy()

drop_cols = [
    "cloud_base_agl:m",
    "ceiling_height_agl:m",
    "snow_density:kgm3",
    "hour",
    "month",
    "date_calc",
    "date_forecast",
    "clear_sky_rad:W",
]

data_general_catboost_transform["pv_measurement"] = data_general_catboost_transform["pv_measurement"]

mean_target = (
    data_general_catboost_transform[
        data_general_catboost_transform.data_type.isin(["observed", "estimated"])
    ]
    .groupby(["location", "month", "hour"])["pv_measurement"]
    .mean()
)

location_mapping = mean_target.to_dict()

data_general_catboost_transform[
    "location_encoded"
] = data_general_catboost_transform.apply(
    lambda x: location_mapping.get((x["location"], x["month"], x["hour"]), None), axis=1
)


data_general_catboost_transform = data_general_catboost_transform.drop(
    drop_cols, axis=1
)

ignore_cols = [
    "location",
    "dew_or_rime:idx",
    "is_day:idx",
    "is_in_shadow:idx",
    "pv_measurement",
    "data_type",
]

cols = [
    col for col in data_general_catboost_transform.columns if col not in ignore_cols
]

data_general_catboost_transform, _, y_scaler = scale_data(
    data_general_catboost_transform, cols=cols
)

# Create training and test data
train = (
    data_general_catboost_transform[
        data_general_catboost_transform["data_type"].isin(["observed", "estimated"])
    ]
    .drop(columns=["data_type"])
    .copy()
)
test = (
    data_general_catboost_transform[
        data_general_catboost_transform["data_type"] == "test"
    ]
    .drop(columns=["data_type"])
    .copy()
)

In [ ]:
pv_measurement = train["pv_measurement"]
train.drop("pv_measurement", axis=1, inplace=True)

cat_features = [0]
model_params = {
    "iterations": 1000,
    "depth": 9,
    "loss_function": "MAE",
    "cat_features": cat_features,
}

# Initialize lists to store the predictions
all_preds = pd.DataFrame()
temp_test_result = []

# Create a CatBoostRegressor instance for this fold
model = catboost.CatBoostRegressor(**model_params)

# Fit the model
model.fit(train, pv_measurement)

# Get predictions for the validation set
preds = model.predict(test)

In [ ]:
test_result["encoded_catboost"] = pd.DataFrame(
    y_scaler.inverse_transform(preds.reshape(-1, 1)), columns=["pv_measurement"]
)
test_result["encoded_catboost"]["pv_measurement"] = test_result[
    "encoded_catboost"
].values

## Model 2 – Simple LightGBM

In [ ]:
data_general_lgb = data_catboost.copy()

drop_cols = [
    "cloud_base_agl:m",
    "ceiling_height_agl:m",
    "snow_density:kgm3",
    "month_sin",
    "hour_sin",
    "month_cos",
    "hour_cos",
    "date_calc",
    "sun_azimuth:d_sin",
    "sun_azimuth:d_cos",
    "date_forecast",
    "elevation:m",
    "precip_5min:mm",
    "precip_type_5min:idx",
    "snow_drift:idx",
    "wind_speed_u_10m:ms",
    "wind_speed_v_10m:ms",
    "wind_speed_w_1000hPa:ms",
]

data_general_lgb = data_general_lgb.drop(drop_cols, axis=1)
data_general_lgb.columns = [
    column.replace(":", "_") for column in data_general_lgb.columns
]

ignore_cols = [
    "location",
    "is_day_idx",
    "is_in_shadow_idx",
    "pv_measurement",
    "data_type",
]

data_general_lgb["is_day_idx"] = data_general_lgb["is_day_idx"].astype("int")
data_general_lgb["is_in_shadow_idx"] = data_general_lgb["is_in_shadow_idx"].astype("int")
data_general_lgb["dew_or_rime_idx"] = data_general_lgb["dew_or_rime_idx"].astype("int")

cols = [col for col in data_general_lgb.columns if col not in ignore_cols]

data_general_lgb, _, y_scaler = scale_data(data_general_lgb, cols=cols)

# location one-hot
data_general_lgb = pd.get_dummies(data_general_lgb, columns=["location"])

# Create training and test data
test = data_general_lgb[data_general_lgb["data_type"].isin(["test"])].drop(columns=["data_type"]).copy()
train = data_general_lgb[data_general_lgb["data_type"].isin(["observed", "estimated"])].drop(columns=["data_type"]).copy()

In [ ]:
pv_measurement = train["pv_measurement"]

model_params = {
    "metric": "MSE",
    "max_depth": 6,
    "num_rounds": 200,
    "colsample_bytree": 0.6,
    "learning_rate": 0.04,
    "num_leaves": 140,
}

# Create a CatBoostRegressor instance for this fold
model = lgb.LGBMRegressor(**model_params)

# Fit the model
model.fit(train.drop(columns=["pv_measurement"]), pv_measurement)

# Get predictions for the validation set
preds = model.predict(test.drop(columns=["pv_measurement"]))

In [ ]:
test_result["general_lightgbm"] = pd.DataFrame(y_scaler.inverse_transform(preds.reshape(-1, 1)), columns=["pv_measurement"])

## Model 3 – Target Log Transform Catboost

In [ ]:
data_general_catboost_transform = data_catboost.copy()


# Drop columns that are not needed
drop_cols = [
    "cloud_base_agl:m",
    "ceiling_height_agl:m",
    "snow_density:kgm3",
    "hour",
    "month",
    "date_calc",
    "date_forecast",
    "clear_sky_rad:W",
    # "sun_azimuth:d",
]

data_general_catboost_transform["pv_measurement"] = np.log1p(
    data_general_catboost_transform["pv_measurement"]
)

data_general_catboost_transform = data_general_catboost_transform.drop(
    drop_cols, axis=1
)

ignore_cols = [
    "location",
    "dew_or_rime:idx",
    "is_day:idx",
    "is_in_shadow:idx",
    "pv_measurement",
    "data_type",
]

cols = [
    col for col in data_general_catboost_transform.columns if col not in ignore_cols
]

data_general_catboost_transform, _, y_scaler = scale_data(
    data_general_catboost_transform, cols=cols
)

# Create training and test data
train = (
    data_general_catboost_transform[
        data_general_catboost_transform["data_type"].isin(["observed", "estimated"])
    ]
    .drop(columns=["data_type"])
    .copy()
)
test = (
    data_general_catboost_transform[
        data_general_catboost_transform["data_type"] == "test"
    ]
    .drop(columns=["data_type"])
    .copy()
)

In [ ]:
pv_measurement = train["pv_measurement"]
train.drop("pv_measurement", axis=1, inplace=True)

cat_features = [0]
model_params = {
    "iterations": 1000,
    "depth": 9,
    "loss_function": "MAE",
    "cat_features": cat_features,
}

# For final predictions
# Create a CatBoostRegressor instance for this fold
model = catboost.CatBoostRegressor(**model_params)

# Fit the model
model.fit(train, pv_measurement)

# Get predictions for the validation set
preds = model.predict(test)

In [ ]:
test_result["transform_catboost"] = pd.DataFrame(
    y_scaler.inverse_transform(preds.reshape(-1, 1)), columns=["pv_measurement"]
)
test_result["transform_catboost"]["pv_measurement"] = np.expm1(
    test_result["transform_catboost"].values
)

## Model 4 – Location Specific Catboost

In [ ]:
catboost_location = data_catboost.copy()

# Drop columns that are not needed
drop_cols = [
    "cloud_base_agl:m",
    "ceiling_height_agl:m",
    "snow_density:kgm3",
    "hour",
    "month",
    "date_calc",
    "date_forecast",
    # "sun_azimuth:d",
]

catboost_location = catboost_location.drop(drop_cols, axis=1)

# transform the target variable by subtracting direct radiation and diffuse radiation:
catboost_location["estimated"] = np.where(
    catboost_location.data_type.isin(["estimated", "test"]), 1, 0
)

# scaling the data
ignore_cols = [
    "location",
    "dew_or_rime:idx",
    "is_day:idx",
    "is_in_shadow:idx",
    "data_type",
    "estimated"
]

cols = [col for col in catboost_location.columns if col not in ignore_cols]

# Create training and test data

train_all = catboost_location[catboost_location.data_type.isin(["observed", "estimated"])].drop(columns="data_type").copy()
test = catboost_location[catboost_location.data_type == "test"].drop(columns="data_type").copy()

In [ ]:
model_params = {
    "iterations": 1000,
    "depth": 9,
    "loss_function": "MAE",
}

final_preds = []

for location in ("A", "B", "C"):
    train = train_all[train_all.location == location].drop(columns=["location"])

    train_pool = catboost.Pool(data=train.drop(columns="pv_measurement"), label=train["pv_measurement"], cat_features=["estimated"])
    test_pool = catboost.Pool(data=test[test.location == location].drop(columns=["pv_measurement", "location"]), cat_features=["estimated"])

    # Create a CatBoostRegressor instance for this fold
    model = catboost.CatBoostRegressor(**model_params)

    # Fit the model
    model.fit(train_pool)

    # Get predictions for the validation set
    preds = model.predict(test_pool)

    # Invert scaling for the validation set predictions
    prediction_df = pd.DataFrame(preds, columns=["prediction"], index=test[test.location == location].index)
    prediction_df["location"] = location

    # Append the predictions to the list
    final_preds.append(prediction_df)

In [ ]:
test_result["location_catboost"] = pd.concat(final_preds).reset_index().sort_values(["location", "index"]).drop(columns="index")

## Model 5 – Location Specific AutoGluon

In [ ]:
data_autogluon = data_catboost.copy()

drop_cols = [
    "ceiling_height_agl:m",
    "cloud_base_agl:m",
    "snow_density:kgm3",
    "elevation:m",
    "snow_drift:idx",
    "date_calc",
    "sun_azimuth:d_sin",
    "sun_azimuth:d_cos",
    "prob_rime:p",
    "fresh_snow_1h:cm",
    "dew_or_rime:idx",
    "fresh_snow_3h:cm",
    "hour",
    "month",
    "hour_sin",
    "hour_cos",
    "month_sin",
    "month_cos",
    "year",
    "day",
    "global_rad"
]

data_autogluon["year"] = data_autogluon["date_forecast"].dt.year
data_autogluon["day"] = data_autogluon["date_forecast"].dt.dayofyear
data_autogluon["global_rad"] = data_autogluon["direct_rad:W"] + data_autogluon["diffuse_rad:W"]


data_autogluon["global_rad_-1"] = (
    data_autogluon.groupby(["location", "year", "day"])["global_rad"].shift(-1).fillna(0)
)

data_autogluon["precip_5min:mm-2"] = (
    data_autogluon.groupby(["location", "year", "day"])["precip_5min:mm"]
    .shift(-2)
    .fillna(0)
)
data_autogluon["rain_water:kgm2-2"] = (
    data_autogluon.groupby(["location", "year", "day"])["rain_water:kgm2"]
    .shift(-2)
    .fillna(0)
)

# avg rainfall per month calculated from estimated/observed
sum_rainfall = (
    data_autogluon.loc[data_autogluon["data_type"].isin(["observed", "estimated"])]
    .groupby("month")["precip_5min:mm"]
    .sum()
    .reset_index()
    .rename(columns={"precip_5min:mm": "sum_rainfall"})
)

data_autogluon = data_autogluon.merge(sum_rainfall, on="month", how="left")

# avg tmp per month calculated from estimated/observed
sum_wind = (
    data_autogluon.loc[data_autogluon["data_type"].isin(["observed", "estimated"])]
    .groupby("month")["wind_speed_10m:ms"]
    .sum()
    .reset_index()
    .rename(columns={"wind_speed_10m:ms": "sum_wind"})
)


data_autogluon = data_autogluon.merge(sum_wind, on="month", how="left")

median_wind_hourly = (
    data_autogluon.loc[data_autogluon["data_type"].isin(["observed", "estimated"])]
    .groupby(["month", "hour"])["wind_speed_10m:ms"]
    .median()
    .reset_index()
    .rename(columns={"wind_speed_10m:ms": "median_wind_hourly"})
)

data_autogluon = data_autogluon.merge(median_wind_hourly, on=["month", "hour"], how="left")


data_autogluon["estimated"] = np.where(
    data_autogluon["data_type"].isin(["estimated", "test"]), "e", "o"
)

data_autogluon = data_autogluon.drop(drop_cols, axis=1)

test = data_autogluon[data_autogluon["data_type"].isin(["test"])].copy().drop(columns=["data_type"])

train, val = train_test_split(
    data_autogluon[data_autogluon["data_type"].isin(["estimated"])],
    test_size=0.40,
    random_state=1111,
)

train = pd.concat([data_autogluon[data_autogluon["data_type"].isin(["observed"])], train])
train = train.drop(columns=["data_type"])
val = val.drop(columns=["data_type"])

In [ ]:
# Initialize and train AutoGluon model
tabular_predictor = TabularPredictor(label="pv_measurement", eval_metric="mae")

predictor_A = tabular_predictor.fit(
    train_data=train[train.location == "A"],
    tuning_data=val[val.location == "A"],
    presets="best_quality",
    auto_stack=True,
    use_bag_holdout=True,
    num_stack_levels=0,
    time_limit=60*60*3.5,
)

In [ ]:
# Initialize and train AutoGluon model
tabular_predictor = TabularPredictor(label="pv_measurement", eval_metric="mae")

predictor_B = tabular_predictor.fit(
    train_data=train[train.location == "B"],
    tuning_data=val[val.location == "B"],
    presets="best_quality",
    auto_stack=True,
    use_bag_holdout=True,
    num_stack_levels=0,
    time_limit=60*60*2.5,
)

In [ ]:
# Initialize and train AutoGluon model
tabular_predictor = TabularPredictor(label="pv_measurement", eval_metric="mae")

predictor_C = tabular_predictor.fit(
    train_data=train[train.location == "C"],
    tuning_data=val[val.location == "C"],
    presets="best_quality",
    auto_stack=True,
    use_bag_holdout=True,
    num_stack_levels=0,
    time_limit=60*60*2.5,
)

In [ ]:
# Make predictions on the test dataset
test_data_nolab = test.drop(columns=["pv_measurement"])

y_pred_A = predictor_A.predict(
    test_data_nolab[test_data_nolab.location == "A"]
)
y_pred_B = predictor_B.predict(
    test_data_nolab[test_data_nolab.location == "B"]
)
y_pred_C = predictor_C.predict(
    test_data_nolab[test_data_nolab.location == "C"]
)

# Add autogluon result to dataframe
autogluon_result = pd.DataFrame(y_pred_A)
autogluon_result["location"] = "A"
autogluon_result = pd.concat([autogluon_result, pd.DataFrame(y_pred_B)], axis=0)
autogluon_result = autogluon_result.fillna("B")
autogluon_result = pd.concat([autogluon_result, pd.DataFrame(y_pred_C)], axis=0)
autogluon_result = autogluon_result.fillna("C")

test_result["autogluon"] = autogluon_result

## Model 6 – Location Specific FastAI Neural Network

In [ ]:
data_neural = data.copy()

# Drop columns that are not needed
drop_cols = [
    "cloud_base_agl:m",
    "ceiling_height_agl:m",
    "snow_density:kgm3",
    "hour",
    "month",
    "date_calc",
    "date_forecast",
    "elevation:m",
    "fresh_snow_12h:cm",
    "year",
    "day",
    "global_rad"
]

# Target transformation
data_neural["pv_measurement"] = data_neural["pv_measurement"]

data_neural["global_rad"] = data_neural["direct_rad:W"] + data_neural["diffuse_rad:W"]


data_neural["year"] = data_neural["date_forecast"].dt.year
data_neural["day"] = data_neural["date_forecast"].dt.dayofyear

data_neural["global_rad_-1"] = (
    data_neural.groupby(["location", "year", "day"])["global_rad"].shift(-1).fillna(0)
)

data_neural["precip_5min:mm-2"] = (
    data_neural.groupby(["location", "year", "day"])["precip_5min:mm"]
    .shift(-2)
    .fillna(0)
)
data_neural["rain_water:kgm2-2"] = (
    data_neural.groupby(["location", "year", "day"])["rain_water:kgm2"]
    .shift(-2)
    .fillna(0)
)
data_neural["precip_5min:mm-2"] = (
    data_neural.groupby(["location", "year", "day"])["precip_5min:mm"]
    .shift(-2)
    .fillna(0)
)

# Agregations

data_neural["total_clouds_rolling_3"] = (
    data_neural.groupby(["location", "year", "day"])["total_cloud_cover:p"]
    .rolling(3)
    .mean()
    .fillna(0)
    .reset_index()["total_cloud_cover:p"]
)
data_neural["precip_5min_rolling_3"] = (
    data_neural.groupby(["location", "year", "day"])["precip_5min:mm"]
    .rolling(3)
    .mean()
    .fillna(0)
    .reset_index()["precip_5min:mm"]
)

# avg rainfall per month calculated from estimated/observed
sum_rainfall = (
    data_neural.loc[data_neural["data_type"].isin(["observed", "estimated"])]
    .groupby("month")["precip_5min:mm"]
    .sum()
    .reset_index()
    .rename(columns={"precip_5min:mm": "sum_rainfall"})
)

data_neural = data_neural.merge(sum_rainfall, on="month", how="left")

# avg tmp per month calculated from estimated/observed
sum_wind = (
    data_neural.loc[data_neural["data_type"].isin(["observed", "estimated"])]
    .groupby("month")["wind_speed_10m:ms"]
    .sum()
    .reset_index()
    .rename(columns={"wind_speed_10m:ms": "sum_wind"})
)


data_neural = data_neural.merge(sum_wind, on="month", how="left")

median_wind_hourly = (
    data_neural.loc[data_neural["data_type"].isin(["observed", "estimated"])]
    .groupby(["month", "hour"])["wind_speed_10m:ms"]
    .median()
    .reset_index()
    .rename(columns={"wind_speed_10m:ms": "median_wind_hourly"})
)

data_neural = data_neural.merge(median_wind_hourly, on=["month", "hour"], how="left")

data_neural["estimated"] = np.where(
    data_neural.data_type.isin(["estimated", "test"]), 1, 0
)

data_neural = data_neural.drop(drop_cols, axis=1)

ignore_cols = [
    "location",
    "dew_or_rime:idx",
    "is_day:idx",
    "is_in_shadow:idx",
    "pv_measurement",
    "data_type",
    "precip_type_5min:idx",
]

cols = [col for col in data_neural.columns if col not in ignore_cols]

train_df = (
    data_neural[data_neural["data_type"].isin(["observed", "estimated"])]
    .copy()
    .drop(columns=["data_type"])
)
test_df = (
    data_neural[data_neural["data_type"] == "test"].copy().drop(columns=["data_type"])
)

In [ ]:
# Assuming you have a DataFrame named 'df' containing your data
dep_var = "pv_measurement"
cat_names = [
    "estimated",
    "dew_or_rime:idx",
    "precip_type_5min:idx",
]  # List of categorical variable names if any
cont_names = [
    col for col in train_df.columns if col not in cat_names + [dep_var, "location"]
]

procs = [
    fastai.tabular.all.Categorify,
    fastai.tabular.all.FillMissing,
    fastai.tabular.all.Normalize,
]  # Data preprocessing steps

all_preds = pd.DataFrame(columns=["prediction"])

importances = {"A": None, "B": None, "C": None}

for location in ("A", "B", "C"):
    fastai.tabular.all.set_seed(42)

    splits = RandomSplitter(seed=42)(
        train_df[train_df.location == location]
        .copy()
        .drop(
            columns=["location"],
            errors="ignore",
        )
    )

    to = fastai.tabular.all.TabularPandas(
        train_df.copy()[train_df.location == location].drop(
            columns=["location"],
            errors="ignore",
        ),
        procs=procs,
        cat_names=cat_names,
        cont_names=[
            col
            for col in train_df.columns
            if col not in cat_names + [dep_var, "location"]
        ],
        y_names=dep_var,
        splits=splits,
        y_block=fastai.tabular.all.RegressionBlock(),
    )

    dls = to.dataloaders(bs=128, seed=42)

    config = tabular_config(ps=0.2)
    # layers=[512, 256, 128, 62],
    learn = fastai.tabular.all.tabular_learner(
        dls,
        # config=config,
        metrics=[fastai.losses.L1LossFlat()],
        loss_func=fastai.losses.L1LossFlat(),
        # layers=[512, 256, 128, 62],
        y_range=(0, train_df[train_df.location == location].pv_measurement.max()),
    )

    lr = learn.lr_find()

    learn.fit_one_cycle(20, lr)

    tst_dl = learn.dls.test_dl(
        test_df[test_df.location == location]
        .copy()
        .drop(
            columns=["location"],
            errors="ignore",
        )
    )
    preds, _ = learn.get_preds(dl=tst_dl)

    all_preds = pd.concat(
        [all_preds, pd.DataFrame(preds.numpy(), columns=["prediction"])]
    )

In [ ]:
test_result["neural_net"] = all_preds

# 3 – Postprocessing

In [ ]:
constraint = (
    data[data["data_type"].isin(["observed"]) & data["month"].isin([5, 6, 7])]
    .groupby(["month", "hour", "location"])["pv_measurement"]
    .max()
    .reset_index()
)

In [ ]:
kaggle_data = (
    data[data.data_type.isin(["test"])]
    .sort_values(["location", "date_forecast"])
    .copy()
).reset_index(drop=True)

kaggle_data["pv_measurement"] = ensamble
kaggle_data["pv_measurement_post"] = pd.merge(
    kaggle_data, constraint, on=["hour", "month", "location"], how="left"
)["pv_measurement_y"].values
kaggle_data["pv_measurement_post"] = np.where(
    kaggle_data["pv_measurement_post"] < kaggle_data["pv_measurement"],
    kaggle_data["pv_measurement_post"],
    kaggle_data["pv_measurement"],
)
kaggle_data["pv_measurement_post"] = np.where(
    kaggle_data["pv_measurement_post"] < 4, 0, kaggle_data["pv_measurement_post"]
)

In [ ]:
np.sum(kaggle_data["pv_measurement"] - kaggle_data["pv_measurement_post"]) / 2160

# 4 – Submission

In [ ]:
output_df = pd.DataFrame(
    kaggle_data["pv_measurement_post"].values, columns=["prediction"]
)
output_df = output_df.reset_index(names="id")
output_df.to_csv(
    "../data/results/" + str(datetime.datetime.now()) + "-ensamble.csv", index=False
)